## Setup and imports 

In [1]:
# --- Imports ---
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import seaborn as sns

# Jupyter display settings
%matplotlib inline
sns.set_style("whitegrid")

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.6.0
CUDA available: False


## Data loading

In [2]:
data_path = '/Users/jaiabhisheksingh/Desktop/DL-Traff-Graph/METRLA/metr-la.h5'  # Adjust path if needed

try:
    df = pd.read_hdf(data_path)
    print("✅ Data loaded successfully!")
    print("Shape:", df.shape)
    display(df.head())
except Exception as e:
    print("❌ Failed to load data:", e)

✅ Data loaded successfully!
Shape: (34272, 207)


,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428571,66.875,59.375000,69.000000,59.250000,69.000000,61.875
2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000,65.000,57.444444,63.333333,...,50.666667,69.875,66.666667,58.555556,62.000,61.111111,64.444444,55.888889,68.444444,62.875
2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857143,62.000
2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000
2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000


## Preprocessing — Normalization and Splitting

In [3]:
from sklearn.preprocessing import StandardScaler

# Convert to numpy
data = df.values
print("Original data shape:", data.shape)

# Normalize
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
print("✅ Data normalized.")

# Convert back to DataFrame for easier handling later
df_normalized = pd.DataFrame(data_normalized, index=df.index, columns=df.columns)


Original data shape: (34272, 207)
✅ Data normalized.


In [4]:
# Load data again
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load traffic data
data_path = '/Users/jaiabhisheksingh/Desktop/DL-Traff-Graph/METRLA/metr-la.h5'  # adjust if needed
df = pd.read_hdf(data_path)

# Normalize
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df.values), index=df.index, columns=df.columns)

# Train/test split
TRAIN_RATIO = 0.8

def create_sequences(data, timestep_in=12, timestep_out=3, mode='TRAIN'):
    xs, ys = [], []
    total_steps = data.shape[0]
    num_train = int(total_steps * TRAIN_RATIO)
    
    if mode == 'TRAIN':
        for i in range(num_train - timestep_in - timestep_out + 1):
            x = data[i:i+timestep_in, :]
            y = data[i+timestep_in+timestep_out-1:i+timestep_in+timestep_out, :]
            xs.append(x)
            ys.append(y)
    elif mode == 'TEST':
        for i in range(num_train - timestep_in, total_steps - timestep_out - timestep_in + 1):
            x = data[i:i+timestep_in, :]
            y = data[i+timestep_in+timestep_out-1:i+timestep_in+timestep_out, :]
            xs.append(x)
            ys.append(y)
    xs, ys = np.array(xs), np.array(ys)
    xs = xs[:, np.newaxis, :, :]  # add channel dim
    ys = ys[:, np.newaxis, :, :]
    return xs, ys

X_train, Y_train = create_sequences(df_normalized.values, mode='TRAIN')
X_test, Y_test = create_sequences(df_normalized.values, mode='TEST')

print(f"✅ X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")


✅ X_train shape: (27403, 1, 12, 207), Y_train shape: (27403, 1, 1, 207)


In [5]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from torchdiffeq import odeint  # Neural ODE solver

# ==== Environment Setup ====
import sys
sys.path.append('./workMETRLA')  # Path where your STGCN_NODE.py is saved

from STGCN_NODE import STGCN  # Import your updated model
from ODEFunc import ODEFunc    # Your ODE function module

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')






In [6]:

# ==== Hyperparameters ====
epochs = 20
learning_rate = 0.001

ks, kt = 3, 3
bs = [[1, 16, 64], [64, 16, 64]]
T = 12
N_NODE = 207
dropout_p = 0.3


In [7]:
# ==== Initialize Model ====
model = STGCN(ks, kt, bs, T, N_NODE, dropout_p).to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
# ==== Initialize Neural ODE ====
A_init = torch.randn((N_NODE, N_NODE), requires_grad=True).to(device)
ode_func = ODEFunc(N_NODE).to(device)
t = torch.tensor([0, 1], dtype=torch.float32).to(device)  # dummy time interval

train_losses = []

In [ ]:
import torch
import torch.optim as optim
from torchdiffeq import odeint
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('./workMETRLA')  # ✅ Your module path

from STGCN_NODE import STGCN
from ODEFunc import ODEFunc

# ==== Hyperparameters ====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

epochs = 20
learning_rate = 0.001
batch_size = 64  # ✅ Batch Training Now

ks, kt = 3, 3
bs = [[1, 16, 64], [64, 16, 64]]
T = 12
N_NODE = 207
dropout_p = 0.3

# ==== Initialize Model and ODE ====
model = STGCN(ks, kt, bs, T, N_NODE, dropout_p).to(device)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ODE for adjacency evolution
A_init = torch.randn((N_NODE, N_NODE), requires_grad=True).to(device)
ode_func = ODEFunc(N_NODE).to(device)
t = torch.tensor([0, 1], dtype=torch.float32).to(device)

train_losses = []

# ==== Training Loop ====
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # === Evolve A once per epoch ===
    A_init_flat = A_init.view(-1)
    A_dynamic_flat = odeint(ode_func, A_init_flat, t)[-1]
    A_dynamic = A_dynamic_flat.view(N_NODE, N_NODE)

    # === NEW: Precompute Lk once ===
    L = scaled_laplacian(A_dynamic.detach().cpu().numpy())
    Lk_dynamic = cheb_poly(L, ks)
    Lk_dynamic = torch.tensor(Lk_dynamic, dtype=torch.float32).to(device)

    n_batches = int(np.ceil(X_train.shape[0] / batch_size))

    for batch_idx in range(n_batches):
        start = batch_idx * batch_size
        end = min((batch_idx + 1) * batch_size, X_train.shape[0])

        x_batch = torch.tensor(X_train[start:end], dtype=torch.float32).to(device)
        y_batch = torch.tensor(Y_train[start:end], dtype=torch.float32).to(device)

        optimizer.zero_grad()
        output = model(x_batch, Lk_dynamic)  # <<< PASS Lk directly
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / n_batches
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")


KeyboardInterrupt: 